In [323]:
from qoop.core import state, ansatz, metric
import qiskit
from qiskit import transpile
from qiskit.quantum_info import Operator, DensityMatrix, Kraus
from scipy.linalg import qr
import numpy as np
import tensorflow as tf

In [324]:
def create_kraus_operators(unitary_matrix):
    '''
        Create a set of Kraus Operators from the input unitary matrix, using QR decomposition
    '''
    kraus_ops = []
    Q, R = qr(unitary_matrix)

    #Q: a 2^N x 2^N matrix, N is the number of qubits
    for q in Q:
        q = np.expand_dims(q, 1)
        kraus_ops.append(q @ np.transpose(np.conjugate(q)))
    return tf.convert_to_tensor(kraus_ops)

def create_unitary_matrix(num_qubits: int):
    """
    Generate a random unitary matrix of size 2^n x 2^n.
    """
    dimension = 2 ** num_qubits
    # Generate a random complex matrix
    random_matrix = np.random.normal(size=(dimension, dimension)) + 0j
    # Perform QR decomposition
    q, _ = qr(random_matrix)
    return q

def calculate_rho2_dephasing(input_rho, num_qubits: int, gamma: float): #for verification
    '''
    Calculate rho2 by dephasing
    '''
    
    # Convert DensityMatrix to numpy array
    rho = input_rho.data
    # Define the Pauli-Z matrix
    sigma_z = np.array([[1, 0], [0, -1]])

    # Calculate the factors
    alpha = 1 + np.sqrt(1 - gamma)
    beta = 1 - np.sqrt(1 - gamma)
    
    # n qubits => qubit thứ n => I @ I @ .... sigma_z (n) @....I
    # Loop for multiple qubits
    for i in range(num_qubits):
        # Create the tensor product of Pauli-Z matrices for all qubits
        sigma_z_i = np.eye(1)
        for j in range(num_qubits):
            if j == i:
                sigma_z_i = np.kron(sigma_z_i, sigma_z)
            else:
                sigma_z_i = np.kron(sigma_z_i, np.eye(2))
        
        # Apply the dephasing formula
        rho = alpha * rho + beta * (sigma_z_i @ rho @ sigma_z_i)
        # Normalize the density matrix (optional, depending on the context)
        rho /= np.trace(rho)

    return rho

def calculate_rho2_from_unitary(rho, unitary_matrix):
    '''
    Calculate rho' by applying U @ rho @ U(dagger)
    '''
    rho_2 = unitary_matrix  @ rho.data @ np.transpose(np.conjugate(unitary_matrix))

    return rho_2


def calculate_rho_3 (rho_2, kraus_operators):
    '''
        sum(K @ rho_2 @ K(dagger)) //rho" = rho
    '''

    rho_3 = sum(K @ rho_2 @ np.transpose(np.conjugate(K)) for K in kraus_operators)
    return rho_3





In [325]:
def compilation_trace_fidelity(rho, sigma):
    """Calculating the fidelity metric

    Args:
        - rho (DensityMatrix): first density matrix
        - sigma (DensityMatrix): second density matrix

    Returns:
        - float: trace metric has value from 0 to 1
    """
    rho_2 = tf.linalg.sqrtm((tf.linalg.sqrtm(rho)) @ (rho))

    # Cast to a supported type
    real_part = tf.math.real(rho_2)
    imaginary_part = tf.math.imag(rho_2)

    # Check for NaNs in both real and imaginary parts
    contains_nan_real = tf.reduce_any(tf.math.is_nan(real_part))
    contains_nan_imag = tf.reduce_any(tf.math.is_nan(imaginary_part))

    contains_nan = contains_nan_real or contains_nan_imag

    if contains_nan == True:
        rho_2 = rho

    return tf.linalg.trace(
            rho_2
            @ (tf.linalg.sqrtm(sigma))
        )
def frobenius_norm(rho, sigma):
    """
    Compute the Frobenius norm between two matrices.

    Parameters:
    rho (numpy.ndarray): The first matrix.
    sigma (numpy.ndarray): The second matrix.

    Returns:
    float: The Frobenius norm between rho and rho_prime.
    """

    # Ensure the matrices are TensorFlow tensors
    rho = tf.convert_to_tensor(rho, dtype=tf.complex128)
    sigma = tf.convert_to_tensor(sigma, dtype=tf.complex128)

    # Compute the difference between the matrices
    diff = rho - sigma

    # Compute the Frobenius norm
    #norm = tf.linalg.normalize(diff, ord='fro')
    norm = tf.sqrt(tf.reduce_sum(tf.square(diff)))
    return norm

#cost func to compare 2 given rhos
def cost(rho, rho_3):
    return tf.square(frobenius_norm(rho, rho_3))
    #return 1 - compilation_trace_fidelity(rho, rho_3)

In [326]:
#Auto Diff
'''
    rho: The initial rho
    unitary_matrix: randomly generated circle in initialization step
    kraus_operators: set of kraus operators
    n: number of qubits
    alpha: learning rate (?)'''
def calculate_derivative(rho, rho2, kraus_operators, num_qubits, alpha=0.1):
    tensorKraus = tf.Variable(kraus_operators)
    with tf.GradientTape() as tape:
        rho3 = calculate_rho_3(rho2, tensorKraus)
        f = cost(rho, rho3) 
    
    # Calculate the gradient
    c = tape.gradient(f, tensorKraus)
    
    # Ensure proper reshaping (ensure this matches the dimensions of your system)
    c = tf.reshape(c, (2**num_qubits, 2**num_qubits, 2**num_qubits))

    # Calculate projection
    proj = c - tensorKraus @ (np.transpose(np.conjugate(c)) @ tensorKraus + np.transpose(np.conjugate(tensorKraus)) @ c) / 2

    # Update the Kraus operators
    updated_kraus_operators = tensorKraus - alpha * proj
    
    # Return the updated Kraus operators
    return updated_kraus_operators



In [327]:
# Adam optimizer function for updating Kraus operators
def calculate_derivative_adam(rho, rho2, kraus_operators, m, v, num_qubits, t, alpha=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
    tensorKraus = tf.Variable(kraus_operators, dtype=tf.complex128)

    beta1 = tf.constant(beta1, dtype=tf.complex128)
    beta2 = tf.constant(beta2, dtype=tf.complex128)
    t = tf.constant(t, dtype=tf.complex128)

    with tf.GradientTape() as tape:
        rho3 = calculate_rho_3(rho2, tensorKraus)
        f = cost(rho, rho3)
    
    # Calculate the gradient
    c = tape.gradient(f, tensorKraus)
    
    # Ensure proper reshaping (ensure this matches the dimensions of your system)
    c = tf.reshape(c, (2**num_qubits, 2**num_qubits, 2**num_qubits))

    # Calculate projection
    proj = c - tensorKraus @ (np.transpose(np.conjugate(c)) @ tensorKraus + np.transpose(np.conjugate(tensorKraus)) @ c) / 2

    # Update Adam variables
    m = beta1 * m + (1 - beta1) * proj
    v = beta2 * v + (1 - beta2) * tf.math.square(proj)

    # Bias correction
    m_hat = m / (1 - tf.pow(beta1, t + 1))
    v_hat = v / (1 - tf.pow(beta2, t + 1))

    # Update the Kraus operators using Adam update rule
    updated_kraus_operators = tensorKraus - alpha * m_hat / (tf.math.sqrt(v_hat) + epsilon)

    return updated_kraus_operators, m, v

In [328]:
def optimize(rho, rho2, kraus_operators, num_qubits, alpha=0.1, num_loop = 1000):
    kraus_operators_copy = tf.identity(kraus_operators)
    # try looping manually
    for i in range (0, num_loop):
        rho3 = calculate_rho_3(rho2, kraus_operators_copy)
        _cost = cost(rho, rho3) 

        #Update Kraus Operators
        kraus_operators_copy=calculate_derivative(rho, rho2, kraus_operators_copy, num_qubits, alpha)

        #Reshape
        kraus_operators_copy = tf.reshape(kraus_operators_copy, (2**num_qubits,2**num_qubits,2**num_qubits))
    
        #print('K(t)K\n',sum(K @ np.transpose(np.conjugate(K)) for K in KrausOperatorsTry)) # I

        print('Loop No.' + str(i) + ': ' + (str)(_cost.numpy()))
        if (_cost.numpy() < 1e-4): 
            break
    return kraus_operators_copy

In [329]:
def optimize_adam(rho, rho2, kraus_operators, num_qubits, alpha=0.1, num_loop = 1000):
    kraus_operators_copy = tf.identity(kraus_operators)
    # Initialize m, v to zero matrices
    m = tf.zeros_like(kraus_operators_copy, dtype=tf.complex128)
    v = tf.zeros_like(kraus_operators_copy, dtype=tf.complex128)
    # try looping manually
    for i in range (0, num_loop):
        rho3 = calculate_rho_3(rho2, kraus_operators_copy)
        _cost = cost(rho, rho3) 

        #Update Kraus Operators
        kraus_operators_copy, m, v =calculate_derivative_adam(rho, rho2, kraus_operators_copy, m, v, num_qubits, i, alpha)

        #Reshape
        kraus_operators_copy = tf.reshape(kraus_operators_copy, (2**num_qubits,2**num_qubits,2**num_qubits))
        m = tf.reshape(m, (2**num_qubits,2**num_qubits,2**num_qubits))
        v = tf.reshape(v, (2**num_qubits,2**num_qubits,2**num_qubits))
        #print('K(t)K\n',sum(K @ np.transpose(np.conjugate(K)) for K in KrausOperatorsTry)) # I

        print('Loop No.' + str(i) + ': ' + (str)(_cost.numpy()))
        if (_cost.numpy() < 1e-4): 
            break
    return kraus_operators_copy

In [330]:
def print_check_unitary(unitary_matrix):
    print("U @ U(dagger)")
    print(unitary_matrix @ np.transpose(np.conjugate(unitary_matrix)))
        
def print_check_kraus(kraus_operators):
    print("sum( K @ K(dagger) )")
    print(sum(K @ np.transpose(np.conjugate(K)) for K in kraus_operators))

In [331]:
def V(num_qubits: int):
    '''
        Create a ansatz V with n qubits and assign its parameters
    '''
    #Assign random parameter
    circuit = ansatz.graph(num_qubits=num_qubits)
    num_params = circuit.num_parameters
    x0 = 2 * np.pi * np.random.random(num_params)
    circuit = circuit.assign_parameters(dict(zip(circuit.parameters, x0)))
    return circuit

In [332]:
num_qubits = 7

# Initial state |0⟩ density matrix for multiple qubits
rho = DensityMatrix.from_label('0' * num_qubits)

# Generate a random unitary_matrix and random set of kraus_operators
unitary_matrix = create_unitary_matrix(num_qubits=num_qubits)
#circuit = V(num_qubits)
# Get the unitary operator corresponding to the circuit
#unitary_op = Operator(circuit)
# Get the unitary matrix
#unitary_matrix = unitary_op.data
# Get first set of kraus_operators
kraus_operators = create_kraus_operators(unitary_matrix=unitary_matrix) 

#print_check_unitary(unitary_matrix=unitary_matrix)
#print_check_kraus(kraus_operators=kraus_operators)

rho2 = calculate_rho2_dephasing(rho, num_qubits, 0.3)


In [333]:
print(rho2)

[[1.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 ...
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]]


In [334]:
# Optimizing
out_kraus_operators = optimize_adam(rho, rho2, kraus_operators, num_qubits, 0.5/(2**num_qubits))

Loop No.0: (0.9821845962619827+0j)
Loop No.1: (0.9529023163830215+0j)
Loop No.2: (0.9236824217695309+0j)
Loop No.3: (0.8903313274828926+0j)
Loop No.4: (0.8518750294244994+0j)
Loop No.5: (0.8077891121275117+0j)
Loop No.6: (0.7581166428468293+0j)
Loop No.7: (0.703262670305392+0j)
Loop No.8: (0.6439158570786883+0j)
Loop No.9: (0.5807828523405713+0j)
Loop No.10: (0.514356608570066+0j)
Loop No.11: (0.44584397142972054+0j)
Loop No.12: (0.37678261370209337+0j)
Loop No.13: (0.30841605522524285+0j)
Loop No.14: (0.24231961678721334+0j)
Loop No.15: (0.1809955136740031+0j)
Loop No.16: (0.1258073870850686+0j)
Loop No.17: (0.07898539632372047+0j)
Loop No.18: (0.04288221590178438+0j)
Loop No.19: (0.018811809890457363+0j)
Loop No.20: (0.007024504218621651+0j)
Loop No.21: (0.007776820701195182+0j)
Loop No.22: (0.017811855897936737+0j)
Loop No.23: (0.03523753192597369+0j)
Loop No.24: (0.05362430113469689+0j)
Loop No.25: (0.06882582512025043+0j)
Loop No.26: (0.07880008047539973+0j)
Loop No.27: (0.0804009

In [335]:
print(1 - cost(rho, calculate_rho_3(rho2, out_kraus_operators)))

#K(dagger)

tf.Tensor((0.9998427442202168+0j), shape=(), dtype=complex128)


In [336]:
print(out_kraus_operators)

tf.Tensor(
[[[ 8.57543464e-02+0.j -2.42679101e-02+0.j  2.27726462e-02+0.j ...
    2.39789061e-02+0.j  2.68698317e-02+0.j -2.37224215e-02+0.j]
  [ 7.35230583e-03+0.j  9.10092551e-02+0.j -1.06519847e-01+0.j ...
   -1.06180484e-01+0.j -1.09505540e-01+0.j  1.06281327e-01+0.j]
  [ 6.21139600e-03+0.j  9.99338498e-02+0.j -1.00165647e-01+0.j ...
   -1.01466666e-01+0.j -8.91660822e-02+0.j  1.01165249e-01+0.j]
  ...
  [ 7.92817852e-03+0.j  1.09132873e-01+0.j -1.09558027e-01+0.j ...
   -1.10617432e-01+0.j -9.81598220e-02+0.j  1.10400891e-01+0.j]
  [ 1.88398199e-03+0.j  7.46306748e-02+0.j -9.23367303e-02+0.j ...
   -9.14706819e-02+0.j -9.44817880e-02+0.j  9.17146764e-02+0.j]
  [-7.12623605e-03+0.j -1.00426566e-01+0.j  1.00536133e-01+0.j ...
    1.00876865e-01+0.j  8.14549172e-02+0.j -1.00816642e-01+0.j]]

 [[ 7.67062877e-02+0.j  4.48223114e-03+0.j  4.66865107e-03+0.j ...
   -4.43992152e-04+0.j  4.35562800e-03+0.j  4.33529404e-03+0.j]
  [-1.33993478e-03+0.j  1.89965544e-02+0.j  1.97866378e-02+0.j .